In [1]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import mysql.connector
 
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Chenyun312!"
)
 
print(mydb)
 
if mydb.is_connected():
    print("CONNECTION SUCCESSFUL")

CONNECTION SUCCESSFUL


In [3]:
#drop a database
#mycursor = mydb.cursor()
#mycursor.execute("DROP DATABASE sqlproject")

In [4]:
#create a database
mycursor = mydb.cursor()

mycursor.execute("CREATE DATABASE sqlproject")

In [5]:
mycursor = mydb.cursor()
mycursor.execute("SHOW DATABASES")

for x in mycursor:
  print(x)

('data1050f24',)
('information_schema',)
('mysql',)
('performance_schema',)
('sqlproject',)
('sqlproject2',)
('sys',)


In [6]:
#create physicians table
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE sqlproject.physicians (SSN VARCHAR(128) PRIMARY KEY, name VARCHAR(255), primary_specialty VARCHAR(128), experience_years INT CHECK(experience_years>=0))")

In [7]:
#create patients table
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE sqlproject.patients (SSN VARCHAR(128) PRIMARY KEY, name VARCHAR(255), address VARCHAR(255), birth_date DATETIME, physician_id VARCHAR(128), foreign key(physician_id) references physicians(SSN))")

In [8]:
#create pharmacies table
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE sqlproject.pharmacies (id VARCHAR(128) PRIMARY KEY, name VARCHAR(255), address VARCHAR(255), phone VARCHAR(128))")

In [9]:
#create drugs table
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE sqlproject.drugs (id VARCHAR(128), name VARCHAR(128) PRIMARY KEY)")

In [10]:
#create prescriptions table
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE sqlproject.prescriptions (id VARCHAR(128) PRIMARY KEY, patient_id VARCHAR(128), physician_id VARCHAR(128), drug_name VARCHAR(128), date DATE, quantity INT, foreign key(patient_id) references patients(ssn), foreign key(physician_id) references physicians(ssn), foreign key(drug_name) references drugs(name))")

In [11]:
#add index
mycursor = mydb.cursor()
mycursor.execute("ALTER TABLE sqlproject.prescriptions ADD INDEX alerting (patient_id, drug_name)")

In [12]:
#create adverse_reactions table
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE sqlproject.adverse_reactions (drug_name VARCHAR(128), drug_name_2 VARCHAR(128), primary key(drug_name, drug_name_2), foreign key(drug_name) references drugs(name))")

In [13]:
#create alerts table
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE sqlproject.alerts (patient_id VARCHAR(128), physician_id VARCHAR(128), alert_date DATE, drug1 VARCHAR(128), drug2 VARCHAR(128), primary key(patient_id, physician_id, alert_date, drug1, drug2), foreign key(patient_id) references patients(ssn), foreign key(physician_id) references physicians(ssn), foreign key(patient_id, drug1) references prescriptions(patient_id, drug_name), foreign key(patient_id, drug2) references prescriptions(patient_id, drug_name), CHECK (drug1<>drug2))")

In [14]:
#create pharmacy_fills table
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE sqlproject.pharmacy_fills (pharmacy_id VARCHAR(128), prescription_id VARCHAR(128), date DATE, cost DECIMAL(8,2), primary key(prescription_id, pharmacy_id), foreign key(prescription_id) references prescriptions(id), foreign key(pharmacy_id) references pharmacies(id))")

In [15]:
#create companies table
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE sqlproject.companies (id VARCHAR(128) PRIMARY KEY, name VARCHAR(128), address VARCHAR(255), contact_phone VARCHAR(128), contact_name VARCHAR(128))")

In [16]:
#create contracts table
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE sqlproject.contracts (Contract_Id VARCHAR(128) PRIMARY KEY, drug VARCHAR(128), dosage INT, pharmacy_id VARCHAR(128), company_id VARCHAR(128), quantity INT, date DATE, price DECIMAL(8,2), foreign key(company_id) references companies(id), foreign key(pharmacy_id) references pharmacies(id), foreign key(drug) references drugs(name))")

In [17]:
#drop patient table
#mycursor = mydb.cursor()
#mycursor.execute("DROP TABLE sqlproject.patients")

In [43]:
#drop physician table
#mycursor = mydb.cursor()
#mycursor.execute("DROP TABLE sqlproject.physicians")

In [15]:
#drop alert table
#mycursor = mydb.cursor()
#mycursor.execute("DROP TABLE sqlproject.alerts")

In [13]:
#drop pharmacy_fills table
#mycursor = mydb.cursor()
#mycursor.execute("DROP TABLE sqlproject.pharmacy_fills")

In [16]:
#drop prescriptions table
#mycursor = mydb.cursor()
#mycursor.execute("DROP TABLE sqlproject.prescriptions")

In [18]:
#connect to a specific database
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Chenyun312!", 
  database = "sqlproject" 
)

In [19]:
mycursor = mydb.cursor()
mycursor.execute("SHOW TABLES")

for x in mycursor:
  print(x)

('adverse_reactions',)
('alerts',)
('companies',)
('contracts',)
('drugs',)
('patients',)
('pharmacies',)
('pharmacy_fills',)
('physicians',)
('prescriptions',)


In [20]:
import pandas as pd

In [21]:
pip install fsspec

In [1]:
#create trigger
#should be run before insertion of data and after connecting to a specific database
trigger="""

DROP TRIGGER IF EXISTS alertings;
CREATE TRIGGER alertings
AFTER INSERT ON prescriptions
FOR EACH ROW
BEGIN
    INSERT INTO alerts(patient_id, physician_id, alert_date, drug1, drug2)
    SELECT DISTINCT NEW.patient_id, NEW.physician_id, NEW.date, prev.drug_name, NEW.drug_name
    FROM prescriptions prev
    JOIN adverse_reactions r ON 
        (r.drug_name = prev.drug_name AND r.drug_name_2 = NEW.drug_name)
        OR (r.drug_name_2 = prev.drug_name AND r.drug_name = NEW.drug_name)
    WHERE prev.patient_id = NEW.patient_id;
END

"""
mycursor.execute(trigger)

In [22]:
data = pd.read_csv("physicians.csv")

In [23]:
for i,row in data.iterrows():
            sql = "INSERT INTO physicians VALUES (%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [24]:
data = pd.read_csv("patients.csv")
data['birthdate'] = pd.to_datetime(data['birthdate'].str.replace('T', ' ').str.replace('Z', ''))
data.head()

,ssn,name,address,birthdate,physician_id
0,478-34-0781,Florance Saiz,7 Fair Oaks Place,1988-11-03 23:25:38,614-57-6885
1,885-94-4721,Merry Di Pietro,1 Old Shore Court,1991-02-07 22:00:41,702-16-8749
2,777-39-3296,Myron Cottem,75875 Fulton Crossing,1986-02-20 04:43:29,718-27-0905
3,227-08-7452,Bearnard Remer,18669 Heffernan Point,2008-01-09 05:34:30,230-12-3219
4,805-15-2755,Roxana Worster,54 Hudson Junction,1982-11-12 18:11:55,163-50-5535


In [25]:
for i,row in data.iterrows():
            sql = "INSERT INTO patients VALUES (%s,%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [26]:
data = pd.read_csv("pharmacies.csv")

In [27]:
for i,row in data.iterrows():
            sql = "INSERT INTO pharmacies VALUES (%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [28]:
data = pd.read_csv("drugs.csv")

In [29]:
for i,row in data.iterrows():
            sql = "INSERT INTO drugs VALUES (%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [30]:
data = pd.read_csv("adverse_reactions.csv")

In [31]:
for i,row in data.iterrows():
            sql = "REPLACE INTO adverse_reactions VALUES (%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [32]:
data = pd.read_csv("prescriptions.csv")
data['date'] = pd.to_datetime(data['date'])
data.head()

,id,patient_id,physician_id,drug_name,date,quantity
0,1,478-34-0781,614-57-6885,Avafoxin,2023-03-11,90
1,2,758-08-7274,118-66-5958,Cleotrana,2023-03-12,10
2,3,758-08-7274,118-66-5958,Primalovir,2023-04-11,20
3,4,758-08-7274,118-66-5958,Glucozepam Amcipentin,2023-05-13,12
4,5,303-13-5928,571-13-9020,Olanzanafine,2023-05-24,25


In [33]:
for i, row in data.iterrows():
    print(tuple(row), len(row))  # Check row length

(1, '478-34-0781', '614-57-6885', 'Avafoxin', Timestamp('2023-03-11 00:00:00'), 90) 6
(2, '758-08-7274', '118-66-5958', 'Cleotrana', Timestamp('2023-03-12 00:00:00'), 10) 6
(3, '758-08-7274', '118-66-5958', 'Primalovir', Timestamp('2023-04-11 00:00:00'), 20) 6
(4, '758-08-7274', '118-66-5958', 'Glucozepam Amcipentin', Timestamp('2023-05-13 00:00:00'), 12) 6
(5, '303-13-5928', '571-13-9020', 'Olanzanafine', Timestamp('2023-05-24 00:00:00'), 25) 6
(6, '303-13-5928', '571-13-9020', 'Primalovir', Timestamp('2023-05-24 00:00:00'), 16) 6
(7, '303-13-5928', '571-13-9020', 'Abnazole Toleluble', Timestamp('2023-05-24 00:00:00'), 5) 6
(8, '478-34-0781', '614-57-6885', 'Avafoxin', Timestamp('2023-06-14 00:00:00'), 60) 6
(9, '303-13-5928', '571-13-9020', 'Glucozepam Amcipentin', Timestamp('2023-06-22 00:00:00'), 3) 6
(10, '501-47-2038', '156-28-1945', 'Cleotrana', Timestamp('2023-07-18 00:00:00'), 20) 6
(11, '777-39-3296', '718-27-0905', 'Dantopex Quixilinum', Timestamp('2023-08-02 00:00:00'), 1) 

In [34]:
for i,row in data.iterrows():
            sql = "REPLACE INTO sqlproject.prescriptions VALUES (%s,%s,%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [125]:
#data = pd.read_csv("SQLproject/alerts.csv")

In [ ]:
#for i,row in data.iterrows():
#            sql = "INSERT INTO alerts VALUES (%s,%s,%s,%s,%s)"
#            mycursor.execute(sql, tuple(row))
#            print("Record inserted")
#            mydb.commit()

In [35]:
data = pd.read_csv("pharmacy_fills.csv")
data['date'] = pd.to_datetime(data['date'])
data.head()

,pharmacy_id,prescription_id,date,cost
0,1,3,2023-03-15,60.53
1,3,4,2023-05-16,41.50
2,1,2,2023-03-12,18.00
3,8,1,2023-03-12,46.53
4,10,5,2023-05-26,47.50


In [36]:
for i,row in data.iterrows():
            sql = "INSERT INTO pharmacy_fills VALUES (%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [37]:
data = pd.read_csv("companies.csv")

In [38]:
for i,row in data.iterrows():
            sql = "INSERT INTO companies VALUES (%s,%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [39]:
data = pd.read_csv("contracts.csv")
data['date'] = pd.to_datetime(data['date'])
data.head()

,Contract_Id,drug,dosage,pharmacy_id,pharm_company_id,quantity,date,price
0,1,Cleotrana,50,5,10,40,2023-10-03,100.0
1,2,Primalovir,500,5,10,20,2023-09-26,40.5
2,3,Kanulin,1000,4,9,20,2023-09-20,10.5
3,4,Olanzanafine,50,6,3,80,2023-09-24,150.0
4,5,Avafoxin,5,15,5,30,2023-09-29,18.0


In [40]:
for i,row in data.iterrows():
            sql = "INSERT INTO contracts VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [ ]:
#create procedure
procedure="""

CREATE PROCEDURE GetPhysicianDetails(IN physician_ssn VARCHAR(128), 
                                    OUT physician_specialty VARCHAR(128),
                                    OUT experience_years INT)
BEGIN
    SELECT primary_specialty, experience_years INTO physician_specialty, experience_years
    FROM physicians
    WHERE physicians.SSN = physician_ssn;
END

"""
mycursor.execute(procedure)